<img width="100" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Forest Emissions Tracking - Validation

_CarbonPlan ClimateTrace Team_

This notebook compares our estimates of country-level forest emissions to prior
estimates from other groups. The notebook currently compares againsts:

- Global Forest Watch (Zarin et al. 2016)
- Global Carbon Project (Friedlingstein et al. 2020)


In [ ]:
import geopandas
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt

from carbonplan_styles.mpl import set_theme

set_theme()

In [ ]:
# Input data
# ----------

# country shapes from GADM36
countries = geopandas.read_file(
    "s3://carbonplan-climatetrace/inputs/shapes/countries.shp"
)

# CarbonPlan's emissions
emissions = {}
versions = ["v0.1", "v0.4"]
for version in versions:
    try:
        emissions[version] = pd.read_csv(
            "s3://carbonplan-climatetrace/{}/country_rollups_emissions.csv".format(
                version
            )
        )
    except:
        emissions[version] = pd.read_csv(
            "s3://carbonplan-climatetrace/{}/country_rollups.csv".format(
                version
            )
        )
        continue

    for mechanism in ["fire", "clearing"]:
        emissions[version + "-" + mechanism] = pd.read_csv(
            "s3://carbonplan-climatetrace/{}/country_rollups_emissions_from_{}.csv".format(
                version, mechanism
            )
        )
    emissions[version] = emissions[version + "-fire"].copy(deep=True)
    emissions[version]["tCO2eq"] = (
        emissions[version + "-fire"]["tCO2eq"]
        + emissions[version + "-clearing"]["tCO2eq"]
    )
# GFW emissions
gfw_emissions = pd.read_excel(
    "s3://carbonplan-climatetrace/validation/gfw_global_emissions.xlsx",
    sheet_name="Country co2 emissions",
).dropna(axis=0)
gfw_emissions = gfw_emissions[
    gfw_emissions["threshold"] == 10
]  # select threshold

# Global Carbon Project
gcp_emissions = (
    pd.read_excel(
        "s3://carbonplan-climatetrace/validation/Global_Carbon_Budget_2020v1.0.xlsx",
        sheet_name="Land-Use Change Emissions",
        skiprows=28,
    )
    .dropna(axis=1)
    .set_index("Year")
)
gcp_emissions *= 3.664  # C->CO2
gcp_emissions.index = [
    pd.to_datetime(f"{y}-01-01") for y in gcp_emissions.index
]
gcp_emissions = gcp_emissions[["GCB", "H&N", "BLUE", "OSCAR"]]

# Blue Sky Fire emissions
emissions["Blue Sky"] = pd.read_csv(
    "s3://carbonplan-climatetrace/validation/forest-fires_bsa.csv"
)

In [ ]:
# Merge emissions dataframes with countries GeoDataFrame
gfw_countries = countries.merge(
    gfw_emissions.rename(columns={"country": "name"}), on="name"
)
trace_countries = {}
for version in versions:
    trace_countries[version] = countries.merge(
        emissions[version].rename(columns={"iso3_country": "alpha3"}),
        on="alpha3",
    )

In [ ]:
# reformat to "wide" format (time x country)
trace_wide = {}
for version in versions:
    trace_wide[version] = (
        emissions[version]
        .drop(columns=["end_date"])
        .pivot(index="begin_date", columns="iso3_country")
        .droplevel(0, axis=1)
    )
    trace_wide[version].index = pd.to_datetime(trace_wide[version].index)

gfw_wide = (
    gfw_emissions.set_index("country")
    .filter(regex="whrc_aboveground_co2_emissions_Mg_.*")
    .T
)
gfw_wide.index = [pd.to_datetime(f"{l[-4:]}-01-01") for l in gfw_wide.index]

gfw_wide.head()

## Part 1 - Compare time-averaged country emissions (tropics only)


In [ ]:
# Create a new dataframe with average emissions
avg_emissions = countries.set_index("alpha3")
for version in versions:
    avg_emissions["trace-" + version] = trace_wide[version].mean().transpose()

avg_emissions = avg_emissions.set_index("name")
avg_emissions["gfw"] = gfw_wide.mean().transpose() / 1e9

In [ ]:
# Scatter Plot
fig, axarr = plt.subplots(ncols=2, sharey=True)
for i, version in enumerate(versions):
    avg_emissions.plot.scatter("gfw", "trace-" + version, ax=axarr[i])
    axarr[i].set_ylabel("Trace [Gt CO2e]")
    axarr[i].set_xlabel("GFW [Gt CO2e]")
    axarr[i].set_title("Trace-{}".format(version))

## Part 2 - Maps of Tropical Emissions


In [ ]:
avg_emissions_nonan = avg_emissions.dropna()

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=2)
for i, version in enumerate(versions):
    kwargs = dict(
        legend=True,
        legend_kwds={
            "orientation": "horizontal",
            "label": "Emissions [Gt CO2e]",
        },
        lw=0.25,
        cmap="Reds",
        vmin=0,
        vmax=1,
        ax=axarr[0, i],
    )
    avg_emissions_nonan.plot("trace-{}".format(version), **kwargs)
    axarr[0, i].set_title("Trace-{}".format(version))

for i, version in enumerate(versions):
    kwargs = dict(
        legend=True,
        legend_kwds={
            "orientation": "horizontal",
            "label": "Emissions Difference [%]",
        },
        lw=0.25,
        cmap="RdBu_r",
        vmin=-40,
        vmax=40,
        ax=axarr[1, i],
    )
    avg_emissions_nonan["pdiff"] = (
        (
            avg_emissions_nonan["trace-{}".format(version)]
            - avg_emissions_nonan["gfw"]
        )
        / avg_emissions_nonan["gfw"]
    ) * 100
    avg_emissions_nonan.plot("pdiff", **kwargs)
    axarr[1, i].set_title("% difference from GFW")

plt.tight_layout()

In [ ]:
kwargs = dict(
    legend=True,
    legend_kwds={"orientation": "horizontal", "label": "Emissions [Gt CO2e]"},
    lw=0.25,
    cmap="Reds",
    vmin=0,
    vmax=1,
    ax=axarr[0, i],
)
avg_emissions_nonan.plot("gfw" ** kwargs)

plt.title("GFW Tropics")

## Part 3 - Compare trace_wideglobal emissions timeseries to Global Carbon Project


In [ ]:
ax = gcp_emissions[["H&N", "BLUE", "OSCAR"]].loc["2000":].plot(ls="--")
gcp_emissions["GCB"].loc["2000":].plot(ax=ax, label="GCB", lw=3)
trace_wide["v0.1"].sum(axis=1).plot(
    ax=ax, label="Trace v0.1".format(version), c="k", lw=3
)
trace_wide["v0.4"].sum(axis=1).plot(
    ax=ax, label="Trace v0.1".format(version), c="k", ls="--", lw=3
)

plt.ylabel("Emissions [Gt CO2e]")
plt.legend()

## Part 4 - Compare fire emissions from CarbonPlan with those from Blue Sky


In [ ]:
comparison = pd.merge(
    emissions["v0.3-fire"].rename({"tCO2eq": "CarbonPlan"}, axis=1),
    emissions["Blue Sky"].rename({"tCO2": "Blue Sky"}, axis=1),
    how="left",
    left_on=["iso3_country", "begin_date"],
    right_on=["iso3_country", "begin_date"],
)

In [ ]:
comparison = comparison.drop(["end_date_x", "end_date_y"], axis=1)
comparison["Blue Sky"] /= 1e9

In [ ]:
for dataset in ["CarbonPlan", "Blue Sky"]:
    comparison = comparison.rename({dataset: dataset + " [GtCO2/year]"}, axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
comparison.plot.scatter(
    "CarbonPlan [GtCO2/year]", "Blue Sky [GtCO2/year]", ax=ax
)
ax.set_xlim(0, 0.9)
ax.set_ylim(0, 0.9)
plt.plot([0, 1], [0, 1], "k-", lw=1)